## Introduction

One of our large global financial clients spends millions of dollars a year on air travel. They believe that some of their contracted airlines are intentionally extending their quoted mileage for routes to lower the results of their cost-per-mile KPI, resulting in them appearing cheaper than the competition. Our company have therefore been tasked to track the point-to-point mileage between their top destinations and compare it to the mileage given by the carriers.

The first step of this is to establish the point-to-point distance between each of the airports given. Our company maintain a master list of airport latitude and longitude provided by the International Air Transport Association, and it is using this that we intend to calculate these distances so that they can be wrapped into a function and compared to the carrier-provided distances live in one of our analytics applications.

#### First, let's have a look at provided dataset...

In [5]:
import pandas as pd
import numpy as np
import geopy.distance
import airportsdata
import re

In [6]:
df = pd.read_csv("Flight-distances.csv")

In [7]:
df.head()

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon
0,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79
1,"Johannesburg, South Africa - London, United Ki...",JNB,LHR,-26.1,28.23,51.47,-0.45
2,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79
3,"Johannesburg, South Africa - London, United Ki...",JNB,LHR,-26.1,28.23,51.47,-0.45
4,"London, United Kingdom - Singapore, Singapore",SIN,LHR,1.3,103.98,51.47,-0.45


#### Calculating distances between airports 

In [8]:
# Calculating distances between airports based on delivered dataset

distance_list = []
for i in df.index:
    departure_coords = (df.Departure_lat[i], df.Departure_lon[i])
    arrival_coords = (df.Arrival_lat[i],df.Arrival_lon[i])
    distance_list.append(round((geopy.distance.geodesic(departure_coords, arrival_coords).km),2))

In [9]:
# Adding to data frame column containing distances between airports

df['Declared_distance_km'] = np.array(distance_list)
df.head()

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Declared_distance_km
0,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
1,"Johannesburg, South Africa - London, United Ki...",JNB,LHR,-26.1,28.23,51.47,-0.45,9040.01
2,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
3,"Johannesburg, South Africa - London, United Ki...",JNB,LHR,-26.1,28.23,51.47,-0.45,9040.01
4,"London, United Kingdom - Singapore, Singapore",SIN,LHR,1.3,103.98,51.47,-0.45,10890.57


#### Creating function allowing us to compare previously calculated and actual distances between airports

In [10]:
# Loading IATA airports data

airports = airportsdata.load('IATA') 
airports = pd.DataFrame.from_dict(airports, 
                                  orient='index', 
                                  columns= ['name', 'city', 'lat', 'lon'])

In [11]:
def checking_distance():

    valid_code = re.compile(r"^[A-Z]{3}$")

    def airport_code(user_input):
        if valid_code.match(user_input.strip()):
            return True
        else:
            return False

    while True:
        user_input = input('Enter Departure Code (Three capital letters according to IATA codes):')
        if airport_code(user_input) == True:
            dep_code_input = user_input
            while True:
                user_input = input('Enter Arrival Code (Three capital letters according to IATA codes):')
                if airport_code(user_input) == True:
                    arr_code_input = user_input
                    break            
                else:
                    print('Invalid code')
            break            
        else:
            print('Invalid code')
            
    
    dep_lon = airports['lon'].loc[dep_code_input]
    dep_lat = airports['lat'].loc[dep_code_input]
    
    arr_lon = airports['lon'].loc[arr_code_input]
    arr_lat = airports['lat'].loc[arr_code_input]
    
    departure_coords = (dep_lat, dep_lon)
    arrival_coords = (arr_lat, arr_lon)
    
      
    print('\nChecking for distance...')
    print('\nProvided flights with calculated distances between airports based on given coordinates:\n')
    print(df[['Normalised City Pair', 'Departure Code', 'Arrival Code', 'Declared_distance_km']].loc[(df['Departure Code'] == dep_code_input) & (df['Arrival Code'] == arr_code_input)].to_string())
    print(f'\nActual distance between airports is: {round((geopy.distance.geodesic(departure_coords, arrival_coords).km),2)} km')
    

#### Let's try our function on the three most frequented flights

In [12]:
df[['Departure Code', 'Arrival Code']].value_counts().head(3)

Departure Code  Arrival Code
LHR             JFK             11
SIN             HKG              9
HKG             SIN              8
dtype: int64

In [13]:
checking_distance()

Enter Departure Code (Three capital letters according to IATA codes): LHR
Enter Arrival Code (Three capital letters according to IATA codes): JFK



Checking for distance...

Provided flights with calculated distances between airports based on given coordinates:

                                             Normalised City Pair Departure Code Arrival Code  Declared_distance_km
0     London, United Kingdom - New York, United States Of America            LHR          JFK               5555.04
2     London, United Kingdom - New York, United States Of America            LHR          JFK               5555.04
395   London, United Kingdom - New York, United States Of America            LHR          JFK               5555.04
519   London, United Kingdom - New York, United States Of America            LHR          JFK               5555.04
573   London, United Kingdom - New York, United States Of America            LHR          JFK               5555.04
767   London, United Kingdom - New York, United States Of America            LHR          JFK               5555.04
777   London, United Kingdom - New York, United States Of America       

In [14]:
checking_distance()

Enter Departure Code (Three capital letters according to IATA codes): SIN
Enter Arrival Code (Three capital letters according to IATA codes): HKG



Checking for distance...

Provided flights with calculated distances between airports based on given coordinates:

                             Normalised City Pair Departure Code Arrival Code  Declared_distance_km
49    Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
57    Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
89    Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
104   Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
133   Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
1235  Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
1469  Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG               2563.39
1487  Hong Kong, Hong Kong - Singapore, Singapore            SIN          HKG       

In [15]:
checking_distance()

Enter Departure Code (Three capital letters according to IATA codes): HKG
Enter Arrival Code (Three capital letters according to IATA codes): SIN



Checking for distance...

Provided flights with calculated distances between airports based on given coordinates:

                             Normalised City Pair Departure Code Arrival Code  Declared_distance_km
168   Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
517   Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
619   Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
1038  Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
1141  Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
1148  Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
2645  Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN               2557.37
2941  Hong Kong, Hong Kong - Singapore, Singapore            HKG          SIN       

#### In every of three main routes checked, declared distance was a little bit higher than the actual distance. Let's dig into our flights dataset 

In [16]:
# Checking for off-scale coordinates

df.describe() 

,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Declared_distance_km
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,31.573267,4.342753,30.692543,3.584463,3739.429787
std,23.254100,74.277086,23.835103,74.942567,3555.952405
min,-37.700000,-157.920000,-41.330000,-157.920000,17.250000
25%,25.300000,-73.790000,22.477500,-73.790000,844.147500
50%,40.100000,2.540000,40.000000,3.320000,2204.830000
75%,49.000000,55.350000,47.550000,55.350000,6026.430000
max,64.000000,153.110000,63.990000,174.810000,17014.670000


In [17]:
# Checking for missing values

df.isnull().value_counts()

Normalised City Pair  Departure Code  Arrival Code  Departure_lat  Departure_lon  Arrival_lat  Arrival_lon  Declared_distance_km
False                 False           False         False          False          False        False        False                   3000
dtype: int64

In [18]:
# Checking for repeating city pairs

df['Normalised City Pair'].value_counts()

London, United Kingdom - New York, United States Of America              24
Hong Kong, Hong Kong - Singapore, Singapore                              17
Hong Kong, Hong Kong - Tokyo, Japan                                      16
Edinburgh, United Kingdom - London, United Kingdom                       15
Boston, United States Of America - New York, United States Of America    15
                                                                         ..
Madrid, Spain - Santiago De Compostela, Spain                             1
Calgary, Canada - Toronto, Canada                                         1
Boston, United States Of America - Doha, Qatar                            1
Austin, United States Of America - Tokyo, Japan                           1
Frankfurt, Germany - Helsinki, Finland                                    1
Name: Normalised City Pair, Length: 1602, dtype: int64

In [19]:
# Examining most frequented route: London, United Kingdom - New York, United States Of America

df.loc[df['Normalised City Pair'] == 'London, United Kingdom - New York, United States Of America']

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Declared_distance_km
0,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
2,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
5,"London, United Kingdom - New York, United Stat...",JFK,LCY,40.6,-73.79,51.50,0.05,5590.82
6,"London, United Kingdom - New York, United Stat...",LCY,JFK,51.5,0.05,40.64,-73.79,5588.04
146,"London, United Kingdom - New York, United Stat...",JFK,LHR,40.6,-73.79,51.47,-0.45,5558.84
395,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
519,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
573,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04
669,"London, United Kingdom - New York, United Stat...",NYC,LHR,40.7,-74.01,51.47,-0.45,5566.44
767,"London, United Kingdom - New York, United Stat...",LHR,JFK,51.5,-0.45,40.64,-73.79,5555.04


#### Several inconsistent JFK coordinates can be spotted over this one city pair. What are the actual JFK coordinates delivered by IATA? Let' check!

In [20]:
airports.loc['JFK']

name    John F Kennedy International Airport
city                                New York
lat                                40.639928
lon                               -73.778693
Name: JFK, dtype: object

#### We should examine every JFK coordinates in our dataset. For start - departure coordinates:

In [21]:
df[['Departure Code', 'Departure_lat', 'Departure_lon']].loc[df['Departure Code'] == 'JFK'].value_counts()

Departure Code  Departure_lat  Departure_lon
JFK              40.6          -73.79           90
                 51.5          -0.45             8
                 25.3           55.35            3
                 35.8           140.39           2
                 50.1           8.56             2
                -23.4          -46.48            1
                 1.3            103.98           1
                 18.4          -66.01            1
                 22.3           113.94           1
                 33.4          -110.87           1
                 34.1          -118.24           1
                 40.8          -111.89           1
                 47.5           8.57             1
                 52.3           4.75             1
dtype: int64

#### As we can see above, JFK airport has a lot of different departure coordinates across dataset. We should take a closer look at some of them. For start - JFK coordinate with lattitude -23,4 and longitude -46,48:

In [22]:
df.loc[(df['Departure Code'] == 'JFK') & (df['Departure_lat'] == -23.4)]

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Declared_distance_km
2807,"Belo Horizonte-confins, Brazil - New York, Uni...",JFK,CNF,-23.4,-46.48,-19.63,-43.96,492.32


#### To proper check distance between this two airports, we have to confirm actual coordinates for CNF (Belo Horizonte) airport:

In [23]:
df.loc[(df['Departure Code'] == 'JFK') & (df['Departure_lat'] == -23.4)]

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Declared_distance_km
2807,"Belo Horizonte-confins, Brazil - New York, Uni...",JFK,CNF,-23.4,-46.48,-19.63,-43.96,492.32


#### To proper check distance between this two airports, we have to confirm actual coordinates for CNF (Belo Horizonte) airport:

In [24]:
airports.loc['CNF']

name    Tancredo Neves International Airport
city                          Belo Horizonte
lat                               -19.624443
lon                               -43.971943
Name: CNF, dtype: object

#### The actual coordinates of CNF and coordinates in dataset are almost identical. We clearly have incorrectly entered coordinates for JFK. What are the consequences of that mistake? We can use our function:

In [26]:
checking_distance()

Enter Departure Code (Three capital letters according to IATA codes): JFK
Enter Arrival Code (Three capital letters according to IATA codes): CNF



Checking for distance...

Provided flights with calculated distances between airports based on given coordinates:

                                                     Normalised City Pair Departure Code Arrival Code  Declared_distance_km
2807  Belo Horizonte-confins, Brazil - New York, United States Of America            JFK          CNF                492.32

Actual distance between airports is: 7350.65 km


#### Actual distance between airports is over 6858 km longer than the distance calculated based on provided data!

#### We can do the same check for another suspicious JFK coordinates:

In [27]:
df.loc[(df['Departure Code'] == 'JFK') & (df['Departure_lat'] == 35.8)]

,Normalised City Pair,Departure Code,Arrival Code,Departure_lat,Departure_lon,Arrival_lat,Arrival_lon,Declared_distance_km
808,"New York, United States Of America - Osaka, Japan",JFK,ITM,35.8,140.39,34.79,135.44,463.92
2143,"Guam, Guam - New York, United States Of America",JFK,GUM,35.8,140.39,13.48,144.75,2510.82


In [28]:
airports.loc['ITM']

name    Osaka International Airport
city                          Osaka
lat                         34.7855
lon                      135.438004
Name: ITM, dtype: object

In [29]:
checking_distance()

Enter Departure Code (Three capital letters according to IATA codes): JFK
Enter Arrival Code (Three capital letters according to IATA codes): ITM



Checking for distance...

Provided flights with calculated distances between airports based on given coordinates:

                                  Normalised City Pair Departure Code Arrival Code  Declared_distance_km
808  New York, United States Of America - Osaka, Japan            JFK          ITM                463.92

Actual distance between airports is: 11135.1 km


In [30]:
airports.loc['GUM']

name    Guam International Airport
city                          Guam
lat                      13.483951
lon                     144.797127
Name: GUM, dtype: object

In [31]:
checking_distance()

Enter Departure Code (Three capital letters according to IATA codes): JFK
Enter Arrival Code (Three capital letters according to IATA codes): GUM



Checking for distance...

Provided flights with calculated distances between airports based on given coordinates:

                                 Normalised City Pair Departure Code Arrival Code  Declared_distance_km
2143  Guam, Guam - New York, United States Of America            JFK          GUM               2510.82

Actual distance between airports is: 12820.63 km


#### We can see a large discrepancies in those distance calculations.

#### Let's check the coordinates for JFK arrivals:

In [33]:
df[['Arrival Code', 'Arrival_lat', 'Arrival_lon']].loc[df['Arrival Code'] == 'JFK'].value_counts()

Arrival Code  Arrival_lat  Arrival_lon
JFK           40.64        -73.79         130
dtype: int64

#### Arrivals coordinates for JFK seems to be ok - there are 130 entries with the same values.

#### As we can see, arrival coordinates for JFK airport are correct, but some of the departure coordinates are wrong.

#### Also, in the row 669 of our dataset, there is NYC departure code, wich don't belong neither to New York or London airports:

In [34]:
df.iloc[669]

Normalised City Pair    London, United Kingdom - New York, United Stat...
Departure Code                                                        NYC
Arrival Code                                                          LHR
Departure_lat                                                        40.7
Departure_lon                                                      -74.01
Arrival_lat                                                         51.47
Arrival_lon                                                         -0.45
Declared_distance_km                                              5566.44
Name: 669, dtype: object

#### Actual airports in New York and London:

In [35]:
airports.loc[(airports['city'] == 'New York') + (airports['city'] == 'London')]

,name,city,lat,lon
YXU,London Airport,London,43.035599,-81.153900
LTN,London Luton Airport,London,51.874699,-0.368333
BQH,London Biggin Hill Airport,London,51.330799,0.032500
LGW,London Gatwick Airport,London,51.148102,-0.190278
LCY,London City Airport,London,51.505299,0.055278
LHR,London Heathrow Airport,London,51.470600,-0.461941
STN,London Stansted Airport,London,51.884998,0.235000
NHT,RAF Northolt,London,51.553001,-0.418167
ISP,Long Island Mac Arthur Airport,New York,40.796136,-73.100665
JFK,John F Kennedy International Airport,New York,40.639928,-73.778693


#### Let's check if there are more non-existing airport name codes in our dataset

In [36]:
airport_list = np.array(airports.index)
airport_list_dep = np.array(df['Departure Code'])
airport_list_arr = np.array(df['Arrival Code'])

In [37]:
# Checking codes that are in Departure codes in dataset but are not listed in IATA database:

for i in np.where(np.isin(airport_list_dep, airport_list, invert=True)):
      print(airport_list_dep[i])

['NYC' 'MIL' 'TCI' 'LON' 'TSE' 'NYC' 'TCI']


In [38]:
# Checking codes that are in Arrival codes in dataset but are not listed in IATA database:

for i in np.where(np.isin(airport_list_arr, airport_list, invert=True)):
    print(airport_list_arr[i])

['TSE' 'MIL' 'LON' 'TSE' 'NYC' 'PAR' 'MCM' 'TSE' 'RIO' 'PAR' 'BJS' 'TSE'
 'MCM']


#### Some of that codes are metropolitan area codes, not the airport codes and some of them are just non-existing codes.

## Summary

As we can see, based only on flights with departure from JFK, there are a lot of incorrect values in provided dataset. Checked examples have shown, that calculated distances between aiports, based on provided coordinates, are usually a little bit longer than expected, but that happens when both airport coordinates are entered correctly. In examined entries with wrong coordinates, calculated declared distance happens to be way shorter than the actual one. Judgments cannot be made at this point, but based on proven examples, it can be assumed that in a situation where the coordinates are correct, the distance declared will be at the limit of statistical error, but incorrectly entered coordinates usually result in a shorter declared distance between airports tha actual, resulting in a larger cost-per-mile and less cost-effectivity of contracted airlines Dataset needs to be examined for departure/arrival codes and coordinates correctnes. 